# Personalized Rea Estate Agent

## Step 1: Setting Up the Python Application

```
pip install -r requirements.txt

requirements.txt
    langchain==0.0.305
    openai==0.28.1
    pydantic>=1.10.12
    pytest>=7.4.0
    sentence-transformers>=2.2.0
    transformers>=4.31.0
    chromadb==0.4.15
    jupyter==1.0.0

In [25]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain.memory import ConversationSummaryMemory, ConversationBufferMemory, CombinedMemory, ChatMessageHistory
from langchain.chains import ConversationChain
from typing import Any, Dict, Optional, Tuple

import requests

Set OpenAI API key to environment variable so that langchain methods can refer them in the environment variables list.

In [26]:
import os
openai_api_key = open("../../api_keys/openai.key", "rt").read()
os.environ["openai_api_key"] = openai_api_key
OpenAI_CHAT_MODEL = "gpt-3.5-turbo"


## Step 2: Generating Real Estate Listings

Generate real estate listings using a Large Language Model. Generate at least 10 listings This can involve creating prompts for the LLM to produce descriptions of various properties. An example of a listing might be:

In [27]:
import openai

def generate_review(prompt, min_len):
    OpenAI_CHAT_MODEL = "gpt-3.5-turbo"
    openai_instruction = [
        {
            "role": "system",
            "content": "You are a real estate agent."},
        {
            "role": "user",
            "content": f"prompt_text = {prompt}"}
        ]

    try:
        response = openai.ChatCompletion.create(
            model = OpenAI_CHAT_MODEL,
            messages = openai_instruction,
            temperature=0.6,
            max_tokens=2000
        )
        review = response.choices[0].message['content'].strip()

    except openai.error.OpenAIError as err:
        print(f"Encountered an error: {err}")

    return review



Sample to generate synthetic sales talk and neighbor description.

In [28]:
location = "Furano, Hokkaido, Japan"

In [29]:
review_len = 40
prompt_text = f"Write a {review_len} word sales talk for a cozy real estate and its equipments in {location}."
generated_review = generate_review(prompt_text, min_len=review_len)
generated_review

'Step into your dream home in Furano, Hokkaido, Japan! This cozy property comes fully furnished with modern appliances, offering the perfect blend of comfort and style. Enjoy breathtaking views of the surrounding nature and experience the ultimate retreat in this charming real estate gem.'

In [30]:
review_len = 30
prompt_text = f"Write a {review_len} word positive statement around the neighbor of a real estate in {location}."
generated_neighbor = generate_review(prompt_text, min_len=review_len)
generated_neighbor

'Experience the serene beauty of Furano, Hokkaido - a picturesque neighborhood filled with charming cafes, stunning lavender fields, and friendly locals. The perfect place to call home.'

In [31]:
entries = 20
area = "Hokkaido, Japan"
prompt_text = f"Generate a csv file of {entries} entries of real estate data with a column head Neighborhood, Price, Bedrooms, Bathrooms, House Size in {area}. Bedrooms and Bathrooms are numberf of it, and the House Size in square of meters. Price in Japanese Yen. Fluctuate number of bachrooms from 1 to 3. Resond just CSV part only."
csv_data = generate_review(prompt_text, min_len=review_len)
csv_data

'Neighborhood,Price,Bedrooms,Bathrooms,House Size\nSapporo,45000000,3,2,150\nOtaru,32000000,2,1,100\nHakodate,60000000,4,3,200\nAsahikawa,28000000,2,1,90\nKushiro,38000000,3,2,120\nObihiro,42000000,3,2,140\nKitami,35000000,2,1,110\nTomakomai,50000000,3,2,160\nWakkanai,26000000,2,1,80\nMuroran,33000000,2,1,105\nChitose,47000000,3,2,155\nIwamizawa,31000000,2,1,95\nAbashiri,40000000,3,2,130\nFurano,44000000,3,2,145\nNayoro,29000000,2,1,85\nRumoi,37000000,3,2,125\nShibetsu,49000000,3,2,165\nNemuro,27000000,2,1,75\nKutchan,36000000,3,2,115'

In [32]:
with open('./HomeAgent.csv', mode='w') as f:
    f.write(csv_data)

In [33]:
import pandas as pd

df = pd.read_csv('./HomeAgent.csv')
df

,Neighborhood,Price,Bedrooms,Bathrooms,House Size
0,Sapporo,45000000,3,2,150
1,Otaru,32000000,2,1,100
2,Hakodate,60000000,4,3,200
3,Asahikawa,28000000,2,1,90
4,Kushiro,38000000,3,2,120
5,Obihiro,42000000,3,2,140
6,Kitami,35000000,2,1,110
7,Tomakomai,50000000,3,2,160
8,Wakkanai,26000000,2,1,80
9,Muroran,33000000,2,1,105


In [34]:
description = []
neibhborhooddescription = []

for index, row in df.iterrows():
    review_len = 40
    location = row['Neighborhood']
    prompt_text = f"Write a {review_len} word sales talk for a cozy real estate and its equipments in {location}. Mention randomly about the amenities equipped."
    description.append(generate_review(prompt_text, min_len=review_len))

    review_len = 30
    prompt_text = f"Write a {review_len} word positive statement around the neighbor of a real estate in {location}. Mention randomly about the transportation lines, schools, shoppings, and leisure facilities nearby. Also mention about degree of urban randomly in its vicinity."
    neibhborhooddescription.append(generate_review(prompt_text, min_len=review_len))

df['Description'] = description
df['Neighborhood Description'] = neibhborhooddescription

In [35]:
df

,Neighborhood,Price,Bedrooms,Bathrooms,House Size,Description,Neighborhood Description
0,Sapporo,45000000,3,2,150,Step into your dream home in Sapporo! This coz...,"""Located in Sapporo, this property boasts exce..."
1,Otaru,32000000,2,1,100,Step into your dream home in Otaru! This cozy ...,Experience the charm of Otaru living with conv...
2,Hakodate,60000000,4,3,200,Step into your dream home in Hakodate! This co...,"Nestled in Hakodate, this property boasts easy..."
3,Asahikawa,28000000,2,1,90,Step into your dream home in Asahikawa! This c...,"Located in Asahikawa, this property boasts exc..."
4,Kushiro,38000000,3,2,120,Step into your dream home in Kushiro! This coz...,"""Located in Kushiro, this property boasts conv..."
5,Obihiro,42000000,3,2,140,"Step into your dream home in Obihiro, where co...",Experience the vibrant neighborhood of Obihiro...
6,Kitami,35000000,2,1,110,"Step into your dream home in Kitami, where coz...","Nestled in Kitami, this vibrant neighborhood b..."
7,Tomakomai,50000000,3,2,160,"Step into your dream home in Tomakomai, where ...","Nestled in Tomakomai, this vibrant neighborhoo..."
8,Wakkanai,26000000,2,1,80,Step into your dream home in Wakkanai! This co...,"Nestled in Wakkanai, this property boasts exce..."
9,Muroran,33000000,2,1,105,Welcome to this cozy real estate in Muroran! T...,"Located in Muroran, this real estate boasts co..."


Store the CSV, so as not generate the file repeated ly.

In [36]:
df.to_csv('./HomeAgent2.csv')

Load it the dataset from a file.

In [37]:
loader = CSVLoader(file_path='./HomeAgent2.csv')
docs = loader.load()

splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = splitter.split_documents(docs)


## Step 3: Storing Listings in a Vector Database

- **Vector Database Setup**: Initialize and configure ChromaDB or a similar vector database to store real estate listings.
- **Generating and Storing Embeddings**: Convert the LLM-generated listings into suitable embeddings that capture the semantic content of each listing, and store these embeddings in the vector database.

In [38]:
# Chroma DB packages
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain

In [39]:
listings = []
for a in docs:
    listings.append(a.page_content)

splitter = CharacterTextSplitter(separator="Listing",
                                chunk_size=1000,
                                chunk_overlap=0,# overlap set to 0
                                length_function=len,
                                is_separator_regex=False, )

chunks = splitter.create_documents(listings)

In [40]:
db = Chroma.from_documents(chunks, OpenAIEmbeddings())

## Step 4: Building the User Preference Interface

Collect buyer preferences, such as the number of bedrooms, bathrooms, location, and other specific requirements from a set of questions or telling the buyer to enter their preferences in natural language. You can hard-code the buyer preferences in questions and answers, or collect them interactively however you'd like, example:

In [41]:
questions = [
                "How big do you want your house to be?",
                "What are 3 most important things for you in choosing this property?",
                "Which amenities would you like?",
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",
            ]
answers = [
                "A comfortable three-bedroom house with a spacious kitchen and a cozy living room.",
                "A quiet neighborhood, good local schools, and convenient shopping options.",
                "A backyard for gardening, a two-car garage, and a modern, energy-efficient heating system.",
                "Easy access to a reliable bus line, proximity to a major highway, and bike-friendly roads.",
                "A balance between suburban tranquility and access to urban amenities like restaurants and theaters."
            ]

In [42]:
llm = OpenAI(openai_api_key=openai_api_key,
            model_name=OpenAI_CHAT_MODEL,
            temperature=0.6,
            max_tokens=2000)

c:\ProgramData\anaconda3\envs\langchain\Lib\site-packages\langchain\llms\openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
c:\ProgramData\anaconda3\envs\langchain\Lib\site-packages\langchain\llms\openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


Buyer Preference Parsing: Implement logic to interpret and structure these preferences for querying the vector database.

In [52]:
history = ChatMessageHistory()
history.add_user_message(f"""You are realter agent that will recommend your customer on their answers to personal questions. Ask user {len(questions)} questions""")
for question, answer in zip(questions, answers):
    history.add_ai_message(question)
    history.add_user_message(answer)

summary_memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="recommendation_summary",
    input_key="input",
    buffer=f"The human answered {len(questions)} questions",
    return_messages=True)


In [62]:
# you could choose to store some of the q/a in memory as well, in addition to original questions
class MementoBufferMemory(ConversationBufferMemory):
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        input_str, output_str = self._get_input_output(inputs, outputs)
        self.chat_memory.add_ai_message(output_str)

history.add_ai_message("""Now I recommend a home we're considering best suited for you from the listings.""")
conversational_memory = MementoBufferMemory(
    chat_memory=history,
    memory_key="questions_and_answers",
    input_key="input"
)

In [45]:
# Combined
memory = CombinedMemory(memories=[conversational_memory, summary_memory])

In [63]:
RECOMMENDER_TEMPLATE = """The following is a friendly conversation between a human and an AI realter.
The AI is follows human instructions and realter recommends for a human based on the human preference.
and human's persona derived from their answers to questions.
{context}
Personal Questions and Answers:
{questions_and_answers}
Summary of Recommendations:
{recommendation_summary}
"""
PROMPT = PromptTemplate(
    input_variables=["context", "recommendation_summary", "questions_and_answers"],
    template=RECOMMENDER_TEMPLATE
)

## Step 5: Searching Based on Preferences

**Semantic Search Implementation**: Use the structured buyer preferences to perform a semantic search on the vector database, retrieving listings that most closely match the user's requirements.

**Listing Retrieval Logic**: Fine-tune the retrieval algorithm to ensure that the most relevant listings are selected based on the semantic closeness to the buyer’s preferences.

## Step 6: Personalizing Listing Descriptions

LLM Augmentation: For each retrieved listing, use the LLM to augment the description, tailoring it to resonate with the buyer’s specific preferences. This involves subtly emphasizing aspects of the property that align with what the buyer is looking for.

Maintaining Factual Integrity: Ensure that the augmentation process enhances the appeal of the listing without altering factual information.

In [64]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(),
    chain_type_kwargs = {"prompt": PROMPT, "verbose": True, "memory": memory},
)

In [65]:
qa.run("Which resident do you recommend?")



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI realter.
The AI is follows human instructions and realter recommends for a human based on the human preference.
and human's persona derived from their answers to questions.
: 10
Neighborhood: Abashiri
Price: 17000000
Bedrooms: 2
Bathrooms: 1
House Size: 100
Description: Step into this charming Abashiri property and experience cozy living at its finest. With modern amenities, spacious rooms, and a serene atmosphere, this home is perfect for creating lasting memories. Don't miss out on this opportunity to make it yours!
Neighborhood Description: Nestled in the charming neighborhood of Abashiri, this real estate offers a peaceful and welcoming atmosphere, perfect for creating lasting memories and enjoying a serene lifestyle.

: 12
Neighborhood: Abashiri
Price: 40000000
Bedrooms: 3
Bathrooms: 2
House Size: 130
Descripti

KeyError: 'input'